# 层和块

我们先回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1842, -0.0374,  0.1876, -0.1508, -0.1395,  0.1610,  0.2765,  0.1215,
         -0.1455,  0.0184],
        [-0.2283,  0.0090,  0.1579, -0.2020, -0.2565,  0.2588,  0.1472,  0.2896,
         -0.1379,  0.1174]], grad_fn=<AddmmBackward0>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[-0.0521,  0.0355, -0.1253,  0.0083, -0.2621,  0.2276, -0.1011,  0.0044,
         -0.2742, -0.1480],
        [ 0.0621,  0.1329, -0.1670,  0.0708, -0.1603,  0.2823, -0.1873,  0.1842,
         -0.2576, -0.0846]], grad_fn=<AddmmBackward0>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1586, -0.0771, -0.1299,  0.1388,  0.1802, -0.1968, -0.0036, -0.0706,
         -0.1354,  0.0355],
        [ 0.2609,  0.1047,  0.1031,  0.2428,  0.1553, -0.0909, -0.0358, -0.0329,
          0.0110,  0.0527]], grad_fn=<AddmmBackward0>)

在前向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0614, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1583, grad_fn=<SumBackward0>)